# Corrección de las imágenes de la serie Borini

Dentro de un mismo tile, se encuentra imágenes con diferente parámetro `Transform` y diferente BBOX.

Solución: Recortar todas las imágenes por el BBOX con la intersección de **todas las tiles**.

I am writting a python script that it's trying to do the following: Crop several images to a common bounding box. The images to clip have different extensions. My assumption is that, if I compute the rasterio.window of each image to the desired bounding box (transformed to the image CRS), and then perform a reproject to obtain the transformation object and dimesion of the image in the desired CRS (the one of the bounding box to clip), ALL the images should the same rasterio metadata. But this is not happening, I obtain slightly different transformation object for each image. How can I solve this issue?

You're very close to a solid solution — the approach makes sense conceptually, but the discrepancy you're seeing in the `transform` objects is likely due to **slight differences in rounding, alignment, pixel resolutions, or image extents**. Here's a breakdown of **why this happens** and how to **solve it reliably**.

---

### ❓ Why You're Seeing Slightly Different Transforms

Even after reprojecting everything to the same CRS and computing the window using `rasterio.windows.from_bounds`, differences can occur due to:

1. **Resolution differences** — Each image might have a slightly different pixel size, and the `from_bounds` operation aligns the bounds to pixel boundaries.
2. **Extent mismatch** — Some images may not fully cover the common bounding box, so the computed window doesn't align perfectly.
3. **Rounding in `from_bounds`** — When you compute a window from geographic bounds, it maps to pixel indices. Floating point precision and rounding will result in small differences.

---

### ✅ Goal: Get All Images with the Same Metadata

If you want all cropped images to **match exactly** in extent, transform, and size, then:

#### ✅ Recommended Solution: Reproject All Images to a Common Grid

Instead of relying on their own internal transform and computing windows, you can enforce the common output grid yourself. Here's the plan:

---

### 🔧 Step-by-Step Solution

#### 1. **Define the target CRS, transform, and shape**

Choose the CRS and bounding box **you want everything to align to**. Define:

```python
from rasterio.transform import from_bounds
import rasterio

# Common bounding box (xmin, ymin, xmax, ymax) in target CRS
target_bounds = (xmin, ymin, xmax, ymax)
target_crs = 'EPSG:32633'  # or whatever your target CRS is

# Define desired resolution
target_resolution = (10, 10)  # e.g., 10x10 meters per pixel

# Compute transform and output shape
width = int((target_bounds[2] - target_bounds[0]) / target_resolution[0])
height = int((target_bounds[3] - target_bounds[1]) / target_resolution[1])

target_transform = from_bounds(*target_bounds, width=width, height=height)
```

#### 2. **Reproject each image to this target**

```python
from rasterio.warp import reproject, Resampling
import numpy as np

def reproject_to_common_grid(src_path, target_crs, target_transform, width, height):
    with rasterio.open(src_path) as src:
        dst_array = np.empty((src.count, height, width), dtype=src.dtypes[0])
        reproject(
            source=rasterio.band(src, list(range(1, src.count + 1))),
            destination=dst_array,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=target_transform,
            dst_crs=target_crs,
            resampling=Resampling.nearest
        )

        # Return array and metadata for saving
        dst_meta = src.meta.copy()
        dst_meta.update({
            "crs": target_crs,
            "transform": target_transform,
            "width": width,
            "height": height
        })
        return dst_array, dst_meta
```

---

### ✅ Result

Every image will be:

- In the same CRS
- Have **exactly** the same `transform`, `width`, and `height`
- Perfectly stackable or diffable
- Cropped to the common bounding box

---

### 🔁 Optional: Save the Reprojected Images

```python
with rasterio.open("output.tif", "w", **dst_meta) as dst:
    dst.write(dst_array)
```

---

Would you like help wrapping this into a CLI script or batch processor? Or do you want to keep it as a function in a larger workflow?

In [77]:
from rasterio.warp import reproject, calculate_default_transform, Resampling
from rasterio.windows import from_bounds
from pathlib import Path
import geopandas as gpd
import numpy as np
import rasterio

In [65]:
ROOT_DIR = Path(r"F:\Borini\harmoPAF\HarmoPAF_time_series")

In [66]:
# The tiles with bbox errors
tiles_with_err = [
    'p5', 'p12', 'p4', 'p8_2', 'p6', 'p3', 'p7_2', 'p21', 'p20', 'p25',
    'p23', 'p7_4', 'p1_2', 'p24', 'p7_3', 'p19_3', 'p16', 'p11_2', 'p19_2']

Se utiliza la función `from_bounds` en lugar de rasterio.mask ya que con esta última los bordes no se solapaban de forma exacta.

In [75]:
def set_window(src, src_meta: dict, bbox):
    """Create the new clipped matrix by window."""
    # Get the window corresponding to the GeoJSON bounds
    window = from_bounds(*bbox, transform=src_meta["transform"])

    # Get the transform for the new (cropped) imag
    out_trns = src.window_transform(window)

    clipped_meta = src_meta.copy()
    clipped_meta.update({
        'transform': out_trns,
        'height': int(window.height),
        'width': int(window.width),
        'compress': 'DEFLATE',
        'predictor': 2
    })

    return (window, clipped_meta)

In [ ]:
def compute_new_trns(src_meta, dst_crs, window):
    """Adapt the image matrix to the correct CRS."""
    # Obtain the clipped image bounds from the window object
    bounds = rasterio.windows.bounds(window, src_meta["transform"])
    # Calculate the transform and new dimensions for the target CRS
    transform, width, height = calculate_default_transform(
        src_meta["crs"], dst_crs, src_meta["width"], src_meta["height"],
        *bounds
    )
    
    # Prepare metadata to update
    out_meta = src_meta.copy()
    out_meta.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    return out_meta

In [ ]:
def fix_image(img_path: Path, target_crs, bbox):
    """Transform the masked array to the target CRS"""
    # Update the output image name
    img_out = Path(img_path.parent, img_path.stem + '_crp.tif')

    with rasterio.open(img_path) as src:
        meta = src.meta
        # Transform the bounding box to the image crs
        i_bbox = bbox.to_crs(meta["crs"]).total_bounds
        window, clipped_meta = set_window(src, src.meta, i_bbox)

        # Create reproyected matrix
        out_meta = compute_new_trns(clipped_meta, target_crs, window)

        with rasterio.open(img_out, "w", **out_meta) as dst:
            for i in range(1, src.count + 1):
                # Read clipped band from original CRS
                src_data = src.read(i, window=window)
                # Create an empty array to reproject the data
                dst_dim = (out_meta["height"], out_meta["width"])
                dst_data = np.empty(dst_dim, dtype=src.dtypes[i-1])

                # Reproject the image
                reproject(
                    source=src_data,
                    destination=dst_data,
                    src_transform=clipped_meta["transform"],
                    src_crs=clipped_meta["crs"],
                    dst_transform=out_meta["transform"],
                    dst_crs=target_crs,
                    resampling=Resampling.bilinear
                )

                # Write the destination raster
                dst.write(dst_data, i)

In [120]:
def reproject_to_common_grid(
        src_path, target_crs, dst_trns, width, height):
    
    with rasterio.open(src_path) as src:
        dst_array = np.empty(
            (src.count, height, width), dtype=src.dtypes[0])
        
        reproject(
            source=rasterio.band(src, list(range(1, src.count + 1))),
            destination=dst_array,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=dst_trns,
            dst_crs=target_crs,
            resampling=Resampling.nearest
        )

        # Return array and metadata for saving
        dst_meta = src.meta.copy()
        dst_meta.update({
            "crs": target_crs,
            "transform": dst_trns,
            "width": width,
            "height": height
        })
        
        return dst_array, dst_meta

In [123]:
def common_grid_params(src, dst_bbox: list, dst_crs):
    # Get the desired resolution (the one capture from the first image)
    trns = src.transform
    target_res = (trns.a, trns.e) # x and y res
    # Compute transform and output shape for the output raster
    width = abs(int((dst_bbox[2] -dst_bbox[0]) / target_res[0]))
    height = abs(int((dst_bbox[3] -dst_bbox[1]) / target_res[1]))

    window = from_bounds(*dst_bbox, transform = trns)
    dst_trns, width, height = calculate_default_transform(
        src.crs, dst_crs, window.width, window.height, *dst_bbox
    )

    return (dst_trns, width, height)

In [127]:
target_crs = "EPSG:32630"

# Iterar cada tile con errores en los BBOX
for tile in tiles_with_err:
    if tile in ["p5"]:
        continue
    
    print(f"Tile {tile}")
    # Listar todas las imagenes
    tile_path = Path(ROOT_DIR, tile, "5_Harmonized")
    images = tile_path.glob("*.tif")

    # Define the common bbox to all the image inside the tile
    bbox_path = Path(tile_path, "intersect_bbox.geojson")
    bbox = gpd.read_file(bbox_path).to_crs(target_crs).total_bounds
    
    # Store target grid data
    dst_grid = False

    # Reproject each image from above target
    for img_path in images:
        
        if img_path.stem.endswith("_crp"):
            continue

        if type(dst_grid) == type(False):
            # Compute the target grid obtaining the first image res
            with rasterio.open(img_path) as src:
                dst_grid = common_grid_params(src, bbox, target_crs)

        dst_trns, width, height = dst_grid
        dst_arr, dst_meta = reproject_to_common_grid(
            img_path, target_crs, dst_trns, width, height)

        # Update the output image name
        img_out = Path(img_path.parent, img_path.stem + '_crp.tif')
        with rasterio.open(img_out, "w", **dst_meta) as dst:
            dst.write(dst_arr)

Tile p12
Tile p4
Tile p8_2
Tile p6
Tile p3
Tile p7_2
Tile p21
Tile p20
Tile p25
Tile p23
Tile p7_4
Tile p1_2
Tile p24
Tile p7_3
Tile p19_3
Tile p16
Tile p11_2
Tile p19_2
